In [19]:
import pandas as pd
from pandas.core.arrays.period import timedelta
import numpy as np

import os

from pprint import pprint

Loading train/validation files

In [20]:
train = pd.read_excel('./data/train_data/train.xlsx')
val = pd.read_excel('./data/val_data/validate.xlsx')

Reshaping data and formatting dates (function)

In [21]:
def create_avg_day(df):
    if 'avg_day' not in df.columns:
        df['avg_day']=df.iloc[:, 1:].mean(axis = 1)
    else:
        print("There's already a column avg_day! No need to create another one.")
        
        
def rename_prices(df):
    if 'PRICES' in df.columns:
        df.rename(columns={"PRICES": "datetime"}, inplace = True)
    else:
        print("There's no column PRICES.")


def dataformatting(df):
    #wide to long
    df = df.melt(id_vars=['datetime'], value_vars=df.columns[1:25]).sort_values(['datetime', 'variable'])
    df.reset_index(inplace=True, drop=True)
    
    #creating master time column, ulgy but works
    time = df['datetime'].copy()
    for d in range(len(df['datetime'])):
        time[d] = df['datetime'][d]+timedelta(hours = d%24) #decided not to go for the +1, so hour 1 is midnight, makes more sense, now it ends in 2009, otherwise the last measurement was 01.01.2010 00:00:00
    df['time'] = time
    
    #hour from string to int
    df['variable'] = df['variable'].map(lambda x:int(x[-2:]))
    
    #renaming, shullfing columns (not important)
    df.rename(columns={"datetime": "date", "variable": "hour", "value":"price"}, inplace = True)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    
    return df

In [22]:
rename_prices(train)
rename_prices(val)

Discretization functions for tabular q_learning

In [23]:
def discretize_col(in_df:pd.DataFrame,col_name:str,n_bins:int,band_length:float) -> pd.DataFrame:
    out_df = in_df.copy()
    max_band = band_length*n_bins
    bins = np.linspace(0,max_band,n_bins)
    bins = np.concatenate([bins,[np.inf]])
    print(bins)
    new_col = pd.cut(out_df[col_name],bins,labels=False,include_lowest=True)+1
    out_df[col_name] = new_col
    return out_df

def create_df_discrete(in_df:pd.DataFrame, is_val:bool=False) -> pd.DataFrame:
    out_df = dataformatting(in_df)
    out_df['month'] = pd.DatetimeIndex(out_df['date']).month
    #out_df['price_nondiscrete'] = out_df['price']
    if(is_val):
        out_df = discretize_col(out_df,'price',15,11)
    else:
        out_df = discretize_col(out_df,'price',15,10)
    #return out_df[['price','hour','month','price_nondiscrete']]
    return out_df[['price','hour','month']]

Creating discrete dataframes and storing them

In [24]:
categorical_train = create_df_discrete(train)
categorical_val = create_df_discrete(val,is_val=True)

[  0.          10.71428571  21.42857143  32.14285714  42.85714286
  53.57142857  64.28571429  75.          85.71428571  96.42857143
 107.14285714 117.85714286 128.57142857 139.28571429 150.
          inf]
[  0.          11.78571429  23.57142857  35.35714286  47.14285714
  58.92857143  70.71428571  82.5         94.28571429 106.07142857
 117.85714286 129.64285714 141.42857143 153.21428571 165.
          inf]


In [25]:
categorical_train['price'].value_counts(dropna=False)

4     5129
3     4655
5     3818
6     2725
2     2270
7     2133
8     1676
9     1099
1      965
10     623
11     394
12     316
15     249
13     143
14     109
Name: price, dtype: int64

In [26]:
pprint(categorical_train.tail())
pprint(categorical_val.tail())

       price  hour  month
26299      4    20     12
26300      3    21     12
26301      3    22     12
26302      3    23     12
26303      3    24     12
       price  hour  month
17515      3    20     12
17516      3    21     12
17517      3    22     12
17518      3    23     12
17519      3    24     12


In [27]:
train_discrete_path = os.path.join(os.getcwd(),'data/train_data/train_discrete.npy')
val_discrete_path = os.path.join(os.getcwd(),'data/val_data/val_discrete.npy')

with open(train_discrete_path,'wb') as f:
    np.save(f,categorical_train.to_numpy())

with open(val_discrete_path,'wb') as f:
    np.save(f,categorical_val.to_numpy())